In [ ]:
import logging
import os
import sys

import gmpy2 as gmp

# from bds.bounds_utils import find_equivalence_classes
import numpy as np
from gmpy2 import mpfr, mpz
from logzero import logger

from bds.bb import BranchAndBoundNaive  # , BranchAndBoundV1, BranchAndBoundV0
from bds.gf2 import GF
from bds.meel import approx_mc2, approx_mc2_core
from bds.rule import Rule
from bds.sat.min_freq import construct_min_freq_program
from bds.sat.solver import construct_solver
from bds.utils import bin_array, bin_random, mpz_set_bits, randints, solutions_to_dict
from tests.utils import assert_close_mpfr, assert_dict_allclose

%cd ..



logger.setLevel(logging.DEBUG)

num_pts = 1000

ub = 0.8
lmbd = 0.1

delta = 0.8
eps = 0.8

show_progres = True
rand_seed = 1234

## This notebook for testing purposes, it only helps with pen-and-paper computation for testing bounds

In [ ]:
os.chdir("/u/50/ciaperm1/unix/Desktop/sampling-rashomon-decision-set-code")

## For end2end test 

In [ ]:
rules = [
    Rule(
        id=1,
        name="rule-1",
        cardinality=1,
        # truthtable=np.array([0, 1, 0, 1, 0], dtype=bool),
        truthtable=mpz_set_bits(mpz(), [1, 3]),
    ),
    Rule(
        id=2,
        name="rule-2",
        cardinality=1,
        # truthtable=np.array([0, 0, 1, 0, 1], dtype=bool),
        truthtable=mpz_set_bits(mpz(), [2, 4]),
    ),
    Rule(
        id=3,
        name="rule-3",
        cardinality=1,
        # truthtable=np.array([1, 0, 1, 0, 1], dtype=bool),
        truthtable=mpz_set_bits(mpz(), [0, 2, 4]),
    ),
    Rule(
        id=4,
        name="rule-4",
        cardinality=1,
        # truthtable=np.array([1, 0, 1, 0, 1], dtype=bool),
        truthtable=mpz_set_bits(mpz(), [0, 2, 4]),
    ),
]


y = np.array([0, 1, 0, 0, 0], dtype=bool)

X = np.array([[1, 0, 0], [0, 1, 0], [1, 0, 1], [0, 1, 0], [1, 0, 1]])

In [ ]:
ub = float("inf")
lmbd = mpfr(lmbd)

# the first iteration of the branch and bound
bb = BranchAndBoundNaive(rules, ub=ub, y=y, lmbd=lmbd)
first_elb, data_points2rules, equivalence_classes = find_equivalence_classes(y, rules)
bb.reset(first_elb)
node, not_captured = bb.queue.pop()

In [ ]:
data_points2rules

In [ ]:
data_points2rules[1]

In [ ]:
example_rule_list = data_points2rules[1]
n = mpz_set_bits(gmp.mpz(), example_rule_list)
equivalence_classes[n].data_points

In [ ]:
equivalence_classes[n].total_positives

In [ ]:
equivalence_classes[n].total_negatives

In [ ]:
equivalence_classes[n].minority_mistakes

In [ ]:
equivalence_classes

In [ ]:
equivalence_classes[mpz(24)].minority_mistakes

In [ ]:
equivalence_classes[mpz(2)].minority_mistakes

In [ ]:
equivalence_classes[mpz(28)].minority_mistakes

In [ ]:
first_elb == -1 / 5

In [ ]:
lmbd = 0.1
for ub in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    print((ub / lmbd) - 1)

In [ ]:
lmbd = 0.1
for ub in [float("inf")]:
    print((ub / lmbd) - 1)

In [ ]:
lmbd = 0.1
ub = float("inf")
bb = BranchAndBoundV0(rules, ub=ub, y=y, lmbd=lmbd)
feasible_solutions = list(bb.run(return_objective=True))
actual = solutions_to_dict(feasible_solutions)

In [ ]:
actual

In [ ]:
expected = {
    (0, 1): mpfr("0.3"),
    (0, 2): mpfr("0.7"),
    (0, 3): mpfr("0.9"),
    (0, 4): mpfr("0.9"),
    (0, 1, 2): mpfr("0.8"),
    (0, 1, 3): mpfr("1.0"),
    (0, 1, 4): mpfr("1.0"),
    (0, 2, 3): mpfr("1.0"),
    (0, 2, 4): mpfr("1.0"),
    (0, 3, 4): mpfr("1.0"),
    (0, 2, 3, 4): mpfr("1.1"),
    (0, 1, 2, 3): mpfr("1.1"),
    (0, 1, 2, 4): mpfr("1.1"),
    (0, 1, 3, 4): mpfr("1.1"),
    (0, 1, 2, 3, 4): mpfr("1.2"),
}

In [ ]:
for ub in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    print("ub " + str(ub))
    print("threshold " + str((ub / lmbd) - 1))
    bb = BranchAndBoundV0(rules, ub=ub, y=y, lmbd=lmbd)
    feasible_solutions = list(bb.run(return_objective=False))
    print(feasible_solutions)
    print(feasible_solutions)

In [ ]:
expected = {
    (0, 1): mpfr("0.3"),
    (0, 2): mpfr("0.7"),
    (0, 3): mpfr("0.9"),
    (0, 4): mpfr("0.9"),
    (0, 1, 2): mpfr("0.8"),
    (0, 1, 3): mpfr("1.0"),
    (0, 1, 4): mpfr("1.0"),
    (0, 2, 3): mpfr("1.0"),
    (0, 2, 4): mpfr("1.0"),
    (0, 3, 4): mpfr("1.0"),
    (0, 2, 3, 4): mpfr("1.1"),
    (0, 1, 2, 3): mpfr("1.1"),
    (0, 1, 2, 4): mpfr("1.1"),
    (0, 1, 3, 4): mpfr("1.1"),
    (0, 1, 2, 3, 4): mpfr("1.2"),
}

In [ ]:
## Branch and boundV1 -- speed up

In [ ]:
lmbd = 0.1
ub = float("inf")

# the first iteration of the branch and bound
bb = BranchAndBoundV1(rules, ub=ub, y=y, lmbd=lmbd)
feasible_solutions = list(bb.run(return_objective=True))
actual = solutions_to_dict(feasible_solutions)

In [ ]:
actual

In [ ]:
assert_dict_allclose(actual, expected)

In [ ]:
2 / 5 + 1 / 5 + 0.1

In [ ]:
lmbd = 0.1
ub = float("inf")

# the first iteration of the branch and bound
bb = BranchAndBoundV1(rules, ub=ub, y=y, lmbd=lmbd)
feasible_solutions = list(bb.run(return_objective=True))
actual = solutions_to_dict(feasible_solutions)

In [ ]:
actual

#### It is correct, let us now consider a more interesting case with more equivalent bound , which actually prunes a lot 

In [ ]:
rules = [
    Rule(
        id=1,
        name="rule-1",
        cardinality=1,
        # truthtable=np.array([0, 1, 0, 1, 0], dtype=bool),
        truthtable=mpz_set_bits(mpz(), [1, 3, 5, 7]),
    ),
    Rule(
        id=2,
        name="rule-2",
        cardinality=1,
        # truthtable=np.array([0, 0, 1, 0, 1], dtype=bool),
        truthtable=mpz_set_bits(mpz(), [2, 4, 8, 9]),
    ),
    Rule(
        id=3,
        name="rule-3",
        cardinality=1,
        # truthtable=np.array([1, 0, 1, 0, 1], dtype=bool),
        truthtable=mpz_set_bits(mpz(), [0, 2, 4, 10]),
    ),
    Rule(
        id=4,
        name="rule-4",
        cardinality=1,
        # truthtable=np.array([1, 0, 1, 0, 1], dtype=bool),
        truthtable=mpz_set_bits(mpz(), [0, 2, 4, 11]),
    ),
]


y = np.array([0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1], dtype=bool)

X = np.array(
    [
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 1],
        [0, 1, 0],
        [1, 0, 1],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 1],
        [0, 1, 0],
        [1, 0, 1],
        [0, 1, 0],
        [1, 0, 1],
    ]
)

In [ ]:
first_elb, data_points2rules, equivalence_classes = find_equivalence_classes(y, rules)

In [ ]:
[y[i] for i in range(len(y)) if i not in [1, 3, 5, 7]]

In [ ]:
for k in equivalence_classes:
    print(k)

In [ ]:
equivalence_classes[mpz(2)].minority_mistakes

In [ ]:
data_points2rules[2]

In [ ]:
data_points2rules[4]

In [ ]:
data_points2rules[8]

In [ ]:
data_points2rules[9]

In [ ]:
mpzOutb = mpz_set_bits(gmp.mpz(), [2])

In [ ]:
mpzOutb

In [ ]:
mpzOutb = mpz_set_bits(gmp.mpz(), [2])

In [ ]:
equivalence_classes

In [ ]:
0.33 - 2 / 12

In [ ]:
assert len(y) == len(X)

In [ ]:
first_elb

In [ ]:
2 / 12 + 0.1

In [ ]:
lmbd = 0.1
ub = 0.7

# the first iteration of the branch and bound
bb = BranchAndBoundV1(rules, ub=ub, y=y, lmbd=lmbd)
feasible_solutions = list(bb.run(return_objective=True))
actual = solutions_to_dict(feasible_solutions)

In [ ]:
actual

In [ ]:
lmbd = 0.1
ub = 0.7

# the first iteration of the branch and bound
bb = BranchAndBoundV0(rules, ub=ub, y=y, lmbd=lmbd)
feasible_solutions = list(bb.run(return_objective=True))
actual = solutions_to_dict(feasible_solutions)

In [ ]:
actual